# Этап 15: Dual-Track — nanoGPT vs Llama 3.2

Теперь мы переходим на «две колеи». Мы будем применять одни и те же математические трюки к нашей учебной модели nanoGPT и к реальной промышленной модели Llama 3.2 (версия 1B).

### Зачем это нужно?
1. **Архитектурные различия:** Увидим, как RoPE (Rotary Embeddings) и GQA (Grouped Query Attention) влияют на чувствительность к сжатию.
2. **Проверка теории:** Работает ли Wanda одинаково хорошо на модели с 10M и 1B параметров?
3. **Практика:** Поймем, как искать нужные слои в сложных иерархиях Hugging Face.

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from src.model import GPTLanguageModel, device

# 1. Загружаем nanoGPT (Учебная модель)
nanogpt = GPTLanguageModel().to(device)
nanogpt.load_state_dict(torch.load('model_ckpt.pt', map_location=device))
print(f"nanoGPT загружена. Параметров: {sum(p.numel() for p in nanogpt.parameters())/1e6:.1f}M")

# 2. Загружаем Llama 3.2 1B (Промышленная модель)
# Примечание: Мы используем модель 1B, чтобы она влезла в память ноутбука
model_id = "meta-llama/Llama-3.2-1B"
try:
    # Вы можете использовать любую небольшую модель, если нет доступа к Llama
    # Например: "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
    llama = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float16).to(device)
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    print(f"Llama 3.2 загружена. Параметров: {sum(p.numel() for p in llama.parameters())/1e9:.1f}B")
except Exception as e:
    print(f"Ошибка загрузки Llama (возможно, нужен логин в HF): {e}")
    print("Используем TinyLlama как замену...")
    llama = AutoModelForCausalLM.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0", torch_dtype=torch.float16).to(device)
    print("TinyLlama загружена.")

### Сравнение структуры слоев

Давайте найдем аналогичные слои в обеих моделях. Нам нужно понять, куда «бить» нашими методами сжатия.

In [ ]:
def print_layer_structure(name, model):
    print(f"\n--- Структура {name} ---")
    # Выведем первые несколько слоев для примера
    for i, (n, p) in enumerate(model.named_modules()):
        if i > 15: break # Ограничим вывод
        print(f"{n:<40} | {type(p).__name__}")

print_layer_structure("nanoGPT", nanogpt)
print_layer_structure("Llama", llama)

print("\nЗаметили разницу? \nIn nanoGPT: blocks.0.attn.c_attn")
print("In Llama: model.layers.0.self_attn.q_proj (отдельные матрицы для Q, K, V!)")

### Задача на этот этап:
Мы реализуем **Pruning (зануление весов)** параллельно для обеих моделей вручную, без использования `peft`.

1. В nanoGPT мы занулим 50% весов в `c_attn`.
2. В Llama мы занулим 50% весов в `q_proj`.

In [ ]:
def manual_prune(layer, amount=0.5):
    with torch.no_grad():
        w = layer.weight.data
        # Простое магнитудное зануление
        threshold = torch.quantile(w.abs(), amount)
        mask = w.abs() > threshold
        layer.weight.data *= mask
        return mask

# 1. Прунинг nanoGPT
nano_layer = nanogpt.blocks[0].attn.c_attn
manual_prune(nano_layer)
print("nanoGPT: Слой attn.c_attn обрезан на 50%")

# 2. Прунинг Llama
llama_layer = llama.model.layers[0].self_attn.q_proj
manual_prune(llama_layer)
print("Llama: Слой layers.0.self_attn.q_proj обрезан на 50%")

print("\nМатематика одна — масштаб разный. Теперь вы готовы применять SOTA-методы к любой модели!")